In [13]:
pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py


  Using cached langchain_nomic-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_community-0.3.12-py3-none-any.whl.metadata (2.9 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached langchainhub-0.1.21-py3-none-any.whl.metadata (659 bytes)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain-0.3.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langgraph-0.2.59-py3-none-any.whl.metadata (15 kB)
  Using cached tavily_python-0.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached gpt4all-2.8.2-py3-none-macosx_10_15_universal2.whl.metadata (4.8 kB)
  Using cached langchain_core-0.3.25-py3-none-any.whl.metadata (6.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)

In [14]:
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"


In [15]:
local_llm = 'llama3'

SPLIT DATA & RETRIEVE DOCUMENT

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document


# URLs list
urls = [
    "https://www.ai-jason.com/learning-ai/how-to-reduce-llm-cost",
    "https://www.ai-jason.com/learning-ai/gpt5-llm",
    "https://www.ai-jason.com/learning-ai/how-to-build-ai-agent-tutorial-3",
]

# Load documents using FireCrawlLoader
api_key = "fc-c9d25ee6154640b3831af13a188a1d67"
docs = [FireCrawlLoader(api_key=api_key, url=url, mode="scrape").load() for url in urls]

# Flatten the list of documents
docs_list = [item for sublist in docs for item in sublist]

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Filter out complex metadata and ensure proper document formatting, Metadata Cleaning
filtered_docs = []
for doc in doc_splits:
    # Ensure the doc is an instance of Document and has a 'metadata' attribute
    if isinstance(doc, Document) and hasattr(doc, "metadata"):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Add to vectorDB/store documents in a vector database
vectorstore = Chroma.from_documents(
    documents=filtered_docs,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(), #Embedding model used
)

# Set up retriever
retriever = vectorstore.as_retriever()  #fetching/


In [26]:
print("Retrieved Document Content:", doc_splits)


Retrieved Document Content: [Document(metadata={'title': 'How to reduce 78%+ of LLM CostThe REAL cost of LLM (And How to reduce 78%+ of Cost) - YouTube', 'description': 'AI Agent, Real cost of LLM', 'ogTitle': 'How to reduce 78%+ of LLM Cost', 'ogDescription': 'AI Agent, Real cost of LLM', 'ogImage': 'https://cdn.prod.website-files.com/647a9c825f099afe643b5c78/65bedbda9969f366afdfad25_tiktok%20(1).jpg', 'ogLocaleAlternate': [], 'og:title': 'How to reduce 78%+ of LLM Cost', 'og:description': 'AI Agent, Real cost of LLM', 'og:image': 'https://cdn.prod.website-files.com/647a9c825f099afe643b5c78/65bedbda9969f366afdfad25_tiktok%20(1).jpg', 'twitter:title': 'How to reduce 78%+ of LLM Cost', 'twitter:description': 'AI Agent, Real cost of LLM', 'twitter:image': 'https://cdn.prod.website-files.com/647a9c825f099afe643b5c78/65bedbda9969f366afdfad25_tiktok%20(1).jpg', 'og:type': 'website', 'twitter:card': 'summary_large_image', 'viewport': ['width=device-width, initial-scale=1', 'width=device-widt

GRADE DOCUMENTS

In [27]:
# Import required modules
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# Initialize the chat model 
llm = ChatOllama(model="llama3:latest", format="json", temperature=0)

# Define the prompt template
prompt = PromptTemplate(
    template="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a grader assessing the relevance of a retrieved document to a user question. 
If the document contains keywords related to the user question, grade it as relevant. 
It does not need to be a stringent test. The goal is to filter out erroneous retrievals. 

Give a binary 'yes' or 'no' score to indicate whether the document is relevant to the question. 
Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. 

<|start_header_id|>user<|end_header_id|>
Here is the retrieved document: 
    
{document}

Here is the user question: {question}

<|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["question", "document"],
)

# Initialize the retrieval grader pipeline
retrieval_grader = prompt | llm | JsonOutputParser()

# Example inputs
question = "How to reduce LLM COSTS?"
# Assuming `retriever` returns a list of documents
docs = retriever.invoke(question)  
doc_txt = docs[1].page_content  # Access the page content of the second document

# Get the grading result
response = retrieval_grader.invoke({"question": question, "document": doc_txt})

# Print the result
print(response)


{'score': 'yes'}


TEXT GENERATION

In [28]:
# Import required modules
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template
prompt = PromptTemplate(
    template="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

<|start_header_id|>user<|end_header_id|>
Question: {question}
Context: {context}

<|start_header_id|>assistant<|end_header_id|>
Answer: """,
    input_variables=["question", "context"],
)

# Initialize the LLM (replace 'local_11m' with the correct model name, e.g., 'llama3:latest')
llm = ChatOllama(model="llama3:latest", temperature=0)

# Define a function to format documents into a single context string
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

# Chain the components
rag_chain = prompt | llm | StrOutputParser()

# Example usage
question = "What are the benefits of using LLMs?"
retrieved_docs = [
    Document(page_content="LLMs can provide concise answers based on large datasets."),
    Document(page_content="They can also improve efficiency in tasks like summarization and translation."),
]

# Format the context
context = format_docs(retrieved_docs)

# Run the chain
response = rag_chain.invoke({"question": question, "context": context})

# Print the response
print(response)


LLMs (Large Language Models) provide concise answers based on large datasets, making them useful for quick information retrieval. They can also improve efficiency in tasks such as summarization and translation, allowing for faster processing of complex texts. Additionally, LLMs can generate human-like text and assist with language-related tasks.


In [32]:
os.environ['TAVILY_API_KEY'] = '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'

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

HALLUCINATION GRADER

In [49]:
llm = ChatOllama(model="llama3:latest", format="json", temperature=0)

# Define the prompt template
prompt = PromptTemplate(
    template="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a grader assessing whether an answer is grounded in / supported by a set of facts.
Give a  score 'YES' or 'NO' to indicate whether the answer is grounded in / supported by the facts.
Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
Example:


<|start_header_id|>user<|end_header_id|>
Here are the facts:

{documents}

Here is the answer:

{generation}

<|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["generation", "documents"],
)

# Create the hallucination grader pipeline
hallucination_grader = prompt | llm | JsonOutputParser()


# Invoke the grader to assess grounding
hallucination_grader.invoke({"documents": docs, "generation": response})

{'score': 0.5}

ANSWER GRADER

In [81]:
### Answer Grader
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)
# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|>|start_header_id|>system<|end_header_id|> You are a grader assessing whether an answer 
    is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. < leot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
\n ----- \n
{generation}
\n ----- \n
Here is the question: {question} <|eot_id|>|start_header_id|>assistant<|end_header_id|›""",
 input_variables= ["generation", "question"],
)
answer_grader = prompt | llm | JsonOutputParser ()
answer_grader. invoke({"question": question, "generation": response})

{'score': 'yes'}

LANG GRAPH  - SETUP STATES and NODES

In [83]:
import langgraph.graph
print(dir(langgraph.graph))


['END', 'Graph', 'MessageGraph', 'MessagesState', 'START', 'StateGraph', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'add_messages', 'graph', 'message', 'state']


In [104]:
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document

## STATE

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The question to answer.
        generation: The LLM-generated response.
        web_search: A flag indicating whether to perform a web search.
        documents: A list of retrieved documents.
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

## NODES

def retrieve(state: GraphState) -> GraphState:
    """
    Retrieve documents from the vectorstore.

    Args:
        state (GraphState): The current graph state.

    Returns:
        GraphState: Updated state with retrieved documents.
    """
    print("--- RETRIEVE ---")

    question = state["question"]

    # Mock retrieval for demonstration
    documents = [
        Document(page_content="This is document 1."),
        Document(page_content="This is document 2."),
    ]

    return {"documents": documents, "question": question}

def generate(state: GraphState) -> GraphState:
    """
    Generate an answer using RAG (Retrieval-Augmented Generation) based on retrieved documents.

    Args:
        state (GraphState): The current graph state.

    Returns:
        GraphState: Updated state with LLM generation.
    """
    print("--- CHECK DOCUMENT RELEVANCE TO QUESTION ---")

    question = state["question"]
    documents = state["documents"]

    # Initialize filtered documents and web search flag
    filtered_docs = []
    web_search = "No"

    # Score each document
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score["score"]

        # Document relevant
        if grade.lower() == "yes":
            print("--- GRADE: DOCUMENT RELEVANT ---")
            filtered_docs.append(d)
        else:
            # Document not relevant
            print("--- GRADE: DOCUMENT NOT RELEVANT ---")
            # Set flag to indicate the need for a web search
            web_search = "Yes"

    # Return the updated state
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def generate(state):
    """
    Generate an answer using RAG (Retrieval-Augmented Generation) based on retrieved documents.

    Args:
        state (dict): The current graph state.

    Returns:
        dict: Updated state with a new key `generation` that contains the LLM-generated response.
    """
    print("--- GENERATE ---")

    # Retrieve the question and documents from the state
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})

    # Return the updated state
    return {
        "documents": documents,
        "question": question,
        "generation": generation
    }

def web_search(state):
    """
    Perform a web search based on the question.

    Args:
        state (dict): The current graph state.

    Returns:
        dict: The updated state with web search results appended to the documents.
    """
    print("--- WEB SEARCH ---")

    # Extract question and documents from the state
    question = state["question"]
    documents = state.get("documents", [])

    # Perform the web search
    docs = web_search_tool.invoke({"query": question})

    # Combine the web search results into a single string
    web_results = "\n".join([d["content"] for d in docs])

    # Create a Document object for the web results
    web_results = Document(page_content=web_results)

    # Append web results to the existing documents
    if documents:
        documents.append(web_results)
    else:
        documents = [web_results]

    # Return the updated state
    return {"documents": documents, "question": question}

def decide_to_generate(state):
    """
    Determines whether to generate an answer or add a web search.

    Args:
        state (dict): The current graph state.

    Returns:
        str: Decision for the next node to call ("websearch" or "generate").
    """
    print("--- ASSESS GRADED DOCUMENTS ---")

    # Extract required information from the state
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    # Check if web search is required
    if web_search == "Yes":
        # All documents have been filtered, need to re-generate a new query
        print("--- DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH ---")
        return "websearch"
    else:
        # Relevant documents are available, so proceed to generate an answer
        print("--- DECISION: GENERATE ANSWER BASED ON RELEVANT DOCUMENTS ---")
        return "generate"
    
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # Invoke hallucination grader
    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        
        # Check question-answering
        print("---GRADE GENERATION VS QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)# web search
workflow.add_node("retrieve", retrieve) # retrieve 
workflow.add_node("grade_documents", grade_documents) #grade documents
workflow.add_node("generate", generate) # generatae



In [105]:
# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")

# Add all conditional edges for `grade_documents` in a single call
workflow.add_conditional_edges(
    "grade_documents", 
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)

# Add the rest of the edges
workflow.add_edge("websearch", "generate")

# Add conditional edges for `generate`
workflow.add_conditional_edges(
    "generate", 
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)


In [107]:
# COMPILE

app = workflow.compile()

#Test
from pprint import pprint

inputs = {"question": "how to save llm cost?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f'Finisehd running:{key}:')
print(value["generation"])


--- RETRIEVE ---
'Finisehd running:retrieve:'
--CHECK • DOCUMENT RELEVANCE TO QUESTION---
--GRADE: DOCUMENT RELEVANT---
--GRADE: DOCUMENT RELEVANT---
--- ASSESS GRADED DOCUMENTS ---
--- DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH ---
'Finisehd running:grade_documents:'
--- WEB SEARCH ---
'Finisehd running:websearch:'
--- GENERATE ---
'Finisehd running:generate:'
To save on Large Language Model (LLM) costs, optimize your prompts to reduce token usage, use model distillation to transfer knowledge from a large model to a smaller one, and leverage fine-tuning for specialization. Additionally, selecting the right model that balances performance and cost can also help reduce expenses.
